## Import Necessary Libraries

In [5]:
# Frameworks for model buliding & visualiztion
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
import tensorflow as tf

In [7]:
import random
import datetime

## Get helper functions

In [8]:
!wget https://raw.githubusercontent.com/Shakhthi/Deep-Learning/main/NLP_helper.py

--2023-01-31 10:19:00--  https://raw.githubusercontent.com/Shakhthi/Deep-Learning/main/NLP_helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 830 [text/plain]
Saving to: ‘NLP_helper.py’

NLP_helper.py       100%[===================>]     830  --.-KB/s    in 0s      

2023-01-31 10:19:00 (21.9 MB/s) - ‘NLP_helper.py’ saved [830/830]



In [9]:
!git clone https://github.com/Shakhthi/Deep-Learning.git

Cloning into 'Deep-Learning'...
remote: Enumerating objects: 414, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 414 (delta 19), reused 0 (delta 0), pack-reused 367
Receiving objects: 100% (414/414), 207.68 MiB | 35.00 MiB/s, done.
Resolving deltas: 100% (136/136), done.
Updating files: 100% (268/268), done.


In [10]:
from NLP_helper import unzip_data

## Get data

In [11]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip


--2023-01-31 10:19:09--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 172.253.114.128, 172.253.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-01-31 10:19:09 (134 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [12]:
unzip_data("nlp_getting_started.zip")

## Explore data

In [13]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [14]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [15]:
# For randomness let's shuffle the train data
train_data_shuffled = train_data.sample(frac=1, random_state=42)
train_data_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [16]:
# is our dataset imbalanced?
(one, zero) = train_data["target"].value_counts()

class_1 = one/len(train_data)*100
class_0 = zero/len(train_data)*100

print(f" class one percentage:  {class_1} \n class zero percentage: {class_0} ")

 class one percentage:  57.03402075397347 
 class zero percentage: 42.96597924602653 


Data set is around 5% imbalanced which is neglectable. but still let's see

In [17]:
# Let's visualize some data
random_index = random.randint(0, len(train_data_shuffled)-5)
for row in train_data_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f" Target: {target} \n Tweet: {text} \n ---")

 Target: 1 
 Tweet: #vaxshill 2 deaths from measles complications in 10 yrs everyone looses their shit. 8 dead from Legionnaires in a month &amp; crickets 
 ---
 Target: 1 
 Tweet: Summer heat drives bobcats to Calgary backyards ~ 3 http://t.co/kEstuUYc4t http://t.co/PzFBb1P1mj 
 ---
 Target: 1 
 Tweet: My baby girls car wreak this afternoon thank God no serious injuries and she was wearing her seatbelt!!!... http://t.co/NJQV45ndS2 
 ---
 Target: 0 
 Tweet: Musician Kalle Mattson Recreates 34 Classic Album Covers in Clever Music Video for 'Avalanche' http://t.co/yDJpOpH1DW 
 ---
 Target: 0 
 Tweet: @noah_anyname That's where the concentration camps and mass murder come in. 
 
EVERY. FUCKING. TIME. 
 ---


## Split data into train & test

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(train_data_shuffled["text"].to_numpy(),
                                                                              train_data_shuffled["target"].to_numpy(),
                                                                              test_size=0.1,
                                                                              random_state=42)

In [20]:
# check the lengths
len(train_sentences), len(train_labels), len(test_sentences), len(test_labels)

(6851, 6851, 762, 762)

In [21]:
# Check first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text Vectorization

In [22]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [23]:
text_vectorizer = TextVectorization(max_tokens=None, #Maximum size of the vocabulary for this layer.
                                standardize="lower_and_strip_punctuation",
                                split="whitespace",
                                ngrams=None,
                                output_mode="int",
                                output_sequence_length=None,
                                pad_to_max_tokens=False)

In [24]:
# Find average number of tokens in he train tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [25]:
max_tweet_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_tweet_length,
                                output_mode="int",
                                output_sequence_length=max_length)

In [26]:
text_vectorizer

In [27]:
# Fit text vectorizer instance to the train data using adapt() method
text_vectorizer.adapt(train_sentences)

In [28]:
# create a sample sentence & token it
sample = ["Data Scientist Mathanbabu Kaliyappan having first salary 1LPA"]
text_vectorizer(sample)

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[688,   1,   1,   1, 652,  97,   1,   1,   0,   0,   0,   0,   0,
          0,   0]])>

In [29]:
# Choose a random sentence from the train data
random_sentence = random.choice(train_sentences)

tv = text_vectorizer(random_sentence)

print(f" random train sentence:\n {random_sentence} \n\n token-vector:\n {tv}")

 random train sentence:
 Wildfire near Columbia River town is 50 percent contained http://t.co/gzGpWSqyMW #FireNews #WA http://t.co/ay49MTYyL8 

 token-vector:
 [ 146  217 3988  499  801    9  717 1494 3976    1    1 1739    1    0
    0]


In [30]:
# Get most frequent words 
words_in_vocublary = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocublary[:5]
last_5_words =words_in_vocublary[-5:]
print(f" Most Frequent 5 words:\n {top_5_words} \n\n Most unused 5 words:\n {last_5_words}")

 Most Frequent 5 words:
 ['', '[UNK]', 'the', 'a', 'in'] 

 Most unused 5 words:
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [31]:
# how long is our vocubulary?
len(words_in_vocublary) # As the max_tokens parameter

10000

### Word Embedding

In [32]:
from tensorflow.keras.layers import Embedding

In [33]:
embedding = Embedding(input_dim=max_tweet_length,
                      output_dim=128,
                      embeddings_initializer="uniform",
                      input_length=max_length) 
embedding

In [34]:
# Get a random tweet from train data
random_sentence = random.choice(train_sentences)
print(f" random sentence: \n {random_sentence} \n")

sample_embedd = embedding(text_vectorizer([random_sentence]))
sample_embedd

 random sentence: 
 The fear of the Lord is the start of knowledge: but the foolish have no use for wisdom and teaching (Amsal 1:7) 



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02053958, -0.04280231, -0.03111218, ...,  0.01349629,
         -0.03694554,  0.02982559],
        [ 0.03305191, -0.01578344, -0.02793779, ..., -0.00658739,
          0.04687952,  0.00265368],
        [ 0.02306772,  0.03892739, -0.04340547, ...,  0.03118905,
          0.02371379, -0.02727597],
        ...,
        [-0.04347567,  0.04469145, -0.03326641, ...,  0.03002623,
          0.04515702, -0.03450473],
        [ 0.0282678 , -0.02488102, -0.04999698, ..., -0.03017178,
          0.01079732, -0.01120857],
        [ 0.03778113, -0.04910668, -0.00508111, ...,  0.02603401,
          0.04952869,  0.01225859]]], dtype=float32)>

In [35]:
sample_embedd[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.02053958, -0.04280231, -0.03111218, -0.0426585 ,  0.0478397 ,
        0.04563801, -0.02270314,  0.02912093, -0.00897133,  0.01926175,
       -0.03694429, -0.00859349,  0.03546355, -0.0339671 , -0.01171859,
        0.01780547, -0.00613209, -0.02213483,  0.02374819, -0.04819087,
        0.01074026,  0.04062713,  0.00112987, -0.03014321, -0.03395387,
       -0.04073064,  0.02333111, -0.02113512,  0.03284133, -0.01695587,
        0.03679682, -0.02023425,  0.01616791,  0.04939883, -0.04174081,
        0.01363338,  0.02042626, -0.01464659, -0.01631986,  0.010602  ,
        0.03403821,  0.01243812, -0.00740757, -0.03048309,  0.04398597,
       -0.0445488 ,  0.04869416,  0.04382389,  0.03083103,  0.02832533,
        0.01992995,  0.02908835,  0.04775793,  0.04307175, -0.01005092,
       -0.01765258,  0.04034403,  0.02683146, -0.04631744, -0.02147546,
       -0.0231859 ,  0.04655201,  0.00870105, -0.03879754, -0.03911859,
        0.008756

## Modelling a text dataset (setting up our modelling experiments)

Now we've got our data in numerical format, let's start building and comparing different models.

* Model 0: Naive Bayes (baseline) - got this from here: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional LSTM
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Word Embedding (feature extractor)
* Model 7: Same as model 6 but using 10% of data

For each of these models, we're going to be following the TensorFlow steps in modelling:
* Construct the model
* Train the model
* Make predictions with the model
* Track prediction evaluation metrics for later comparison

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [37]:
# create tokenisaion model pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # covert thetext to numbers
    ("clf", MultinomialNB())      # model the text convered numbers
    ])

# Fit the model
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [38]:
# Evaluate the model
baseline_score = model_0.score(test_sentences, test_labels)
print(f" Base model achives an accuracy of: {baseline_score*100:.2f}%")

 Base model achives an accuracy of: 79.27%


In [39]:
# predict using test data
y_pred = model_0.predict(test_sentences)
y_pred[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Evaluation metrics



Let's make a function to evaluate our modelling experiment predictions using: 
* Accuracy
* Precision
* Recall
* F1-score

In [40]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [41]:
def model_evaluation(y_true, y_pred):
  """
  Returns a DataFrame of scores
  Args:
        y_true: actual test labels
        y_pred: predictions of the model
  """
  accuracy = accuracy_score(y_true, y_pred)

  precision = precision_score(y_true, y_pred)

  recall = recall_score(y_true, y_pred)

  f1 = f1_score(y_true, y_pred)

  eval_df = pd.DataFrame(data=[accuracy, precision, recall, f1], 
                         index=["accuracy score", "precision score", "recall score", "f1-score"])
  return eval_df

In [42]:
results = model_evaluation(test_labels, y_pred)

In [43]:
results

,0
accuracy score,0.792651
precision score,0.886179
recall score,0.626437
f1-score,0.734007


### Model:1 Simple Dense Model (Feed-Forward Neural Network)

In [44]:
# create a TensorBoard callback (functionized because we need one for each model)
def tensorboard_callback(dir_name, experiment_name):
  
  logs_directory = dir_name+"/"+experiment_name+"/"+datetime.datetime.now().strftime("%d%m%Y-%H%M%S")
  
  TensorBoard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs_directory)
  print(f"Saving TensorBoard logfiles to: {logs_directory}")
  
  return TensorBoard_callback

In [45]:
from tensorflow.keras import layers

In [46]:
inputs = layers.Input(shape=(1,), dtype=tf.string, name="input_layer")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D(name="pooling_layer")(x)
outputs = layers.Dense(1, activation="sigmoid", name="output_layer")(x)
model_1 = tf.keras.Model(inputs, outputs, name="Dense_Model")

In [47]:
model_1.compile(loss = "binary_crossentropy",
                optimizer = "Adam",
                metrics = ["accuracy"])

In [48]:
meta_1 = model_1.fit(x=train_sentences,
            y=train_labels,
            epochs=5,
            validation_data=(test_sentences, test_labels),
            callbacks=[tensorboard_callback(dir_name="model_logs", 
                                            experiment_name="model_1_dense")])

Saving TensorBoard logfiles to: model_logs/model_1_dense/31012023-101911
Epoch 1/5
215/215 [==============================] - 4s 16ms/step - loss: 0.6126 - accuracy: 0.6862 - val_loss: 0.5360 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.4424 - accuracy: 0.8194 - val_loss: 0.4709 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3471 - accuracy: 0.8603 - val_loss: 0.4607 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2843 - accuracy: 0.8904 - val_loss: 0.4638 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2379 - accuracy: 0.9108 - val_loss: 0.4797 - val_accuracy: 0.7782


In [49]:
# Evaluate model
model_1.evaluate(test_sentences, test_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4797 - accuracy: 0.7782


[0.4797130227088928, 0.778215229511261]

In [50]:
pred_probs = model_1.predict(test_sentences)
pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.3770366 ],
       [0.7473684 ],
       [0.99784684],
       [0.16894352],
       [0.10868072],
       [0.9434095 ],
       [0.9075831 ],
       [0.9936651 ],
       [0.96170086],
       [0.32197723]], dtype=float32)

In [51]:
preds = tf.squeeze(tf.round(pred_probs))
preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [52]:
test_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [53]:
model_1_results = model_evaluation(test_labels, preds)
model_1_results

,0
accuracy score,0.778215
precision score,0.807560
recall score,0.675287
f1-score,0.735524


In [54]:
results

,0
accuracy score,0.792651
precision score,0.886179
recall score,0.626437
f1-score,0.734007


In [55]:
# Let's compare the model 1 results with baseline model
np.array(list(results.values)) < np.array(list(model_1_results.values))

array([[False],
       [False],
       [ True],
       [ True]])

### Visualizing learned embeddings

To further understand word embeddings, let's visualize them, to do so, we'll get the weights matrix (embedding matrix) from our embedding layer and visualize it using the Embedding project tool, see the TensorFlow guide for more: https://www.tensorflow.org/tutorials/text/word_embeddings

In [56]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [57]:
model_1.summary()

Model: "Dense_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 pooling_layer (GlobalAverag  (None, 128)              0         
 ePooling1D)                                                     
                                                                 
 output_layer (Dense)        (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non

In [58]:
# Get the weight matrix of embedding layer
# (the weights are the numerical patterns between the text in the training dataset that the model has learned)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [59]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [60]:
# Let's download the saved embeddings locally
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Our visual word embeddings might not look like much to us, but they help our model understand the relationships between words.

For more on a popular type of word embedding and more visual explanations check out the illustrated word2vec: https://jalammar.github.io/illustrated-word2vec/

## Model 2: LSTM

* LSTM = long short-term memory

For more on RNN's/resources to learn, see here: https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb 

We're going to focus on writing the code.

In [61]:
inputs = layers.Input(shape=(1,), dtype=tf.string, name="input_layer")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After Embedding: {x.shape}")
x = layers.LSTM(units=64, activation="tanh")(x)
#x = layers.LSTM(units=64, activation="tanh", return_sequences=True)(x)
print(f"After LSTM: {x.shape}")
#x = layers.GlobalAveragePooling1D(name="pooling_layer")(x)
outputs = layers.Dense(1, activation="sigmoid", name="output_layer")(x)
model_2 = tf.keras.Model(inputs, outputs, name="LSTM_Model")

After Embedding: (None, 15, 128)
After LSTM: (None, 64)


In [62]:
model_2.compile(loss = "binary_crossentropy",
                optimizer = "Adam",
                metrics = ["accuracy"])

meta_2 = model_2.fit(x=train_sentences,
                    y=train_labels,
                    epochs=5,
                    validation_data=(test_sentences, test_labels),
                    callbacks=[tensorboard_callback(dir_name="model_logs", 
                                                    experiment_name="model_2_LSTM")])

Saving TensorBoard logfiles to: model_logs/model_2_LSTM/31012023-101930
Epoch 1/5
215/215 [==============================] - 9s 29ms/step - loss: 0.2225 - accuracy: 0.9194 - val_loss: 0.5741 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 6s 27ms/step - loss: 0.1576 - accuracy: 0.9409 - val_loss: 0.6184 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 6s 27ms/step - loss: 0.1303 - accuracy: 0.9539 - val_loss: 0.6451 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 6s 27ms/step - loss: 0.1061 - accuracy: 0.9587 - val_loss: 0.8567 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0832 - accuracy: 0.9672 - val_loss: 0.9088 - val_accuracy: 0.7808


In [63]:
model_2.summary()

Model: "LSTM_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
______________________________________________

In [64]:
# Evaluate model
model_2.evaluate(test_sentences, test_labels)

24/24 [==============================] - 0s 6ms/step - loss: 0.9088 - accuracy: 0.7808


[0.908782958984375, 0.7808399200439453]

In [65]:
pred_probs = model_2.predict(test_sentences)
pred_probs[:10]

24/24 [==============================] - 1s 6ms/step


array([[1.6925532e-03],
       [6.9522113e-01],
       [9.9973118e-01],
       [3.2351900e-02],
       [5.8106118e-04],
       [9.8477191e-01],
       [3.4072125e-01],
       [9.9977511e-01],
       [9.9956930e-01],
       [3.4009627e-01]], dtype=float32)

In [66]:
preds_2  = tf.squeeze(tf.round(pred_probs))
preds_2[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 0.], dtype=float32)>

In [67]:
model_2_results = model_evaluation(test_labels, preds_2)
model_2_results

,0
accuracy score,0.780840
precision score,0.854902
recall score,0.626437
f1-score,0.723051


In [68]:
# Let's compare the model 1 results with baseline model
np.array(list(results.values)) < np.array(list(model_2_results.values))

array([[False],
       [False],
       [False],
       [False]])

## Model 3: GRU

GRU = Gated recurrent unit (one of the most popular and useful recurrent layer types)

📖 **Resource:** If you want to see the formula for Tanh, a great extension would be to replicate the function in pure TensorFlow, see here: https://ml-cheatsheet.readthedocs.io/en/latest/activation_functions.html#tanh

In [69]:
inputs = layers.Input(shape=(1,), dtype=tf.string, name="input_layer")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After Embedding: {x.shape}")
x = layers.GRU(units=64, activation="tanh")(x)
#x = layers.GRU(units=64, activation="tanh", return_sequences=True)(x)
print(f"After GRU: {x.shape}")
#x = layers.GlobalAveragePooling1D(name="pooling_layer")(x)
outputs = layers.Dense(1, activation="sigmoid", name="output_layer")(x)
model_3 = tf.keras.Model(inputs, outputs, name="GRU_Model")

After Embedding: (None, 15, 128)
After GRU: (None, 64)


In [70]:
model_3.compile(loss = "binary_crossentropy",
                optimizer = "Adam",
                metrics = ["accuracy"])

meta_3 = model_3.fit(x=train_sentences,
                    y=train_labels,
                    epochs=5,
                    validation_data=(test_sentences, test_labels),
                    callbacks=[tensorboard_callback(dir_name="model_logs", 
                                                    experiment_name="model_3_GRU")])

Saving TensorBoard logfiles to: model_logs/model_3_GRU/31012023-102004
Epoch 1/5
215/215 [==============================] - 9s 29ms/step - loss: 0.1707 - accuracy: 0.9253 - val_loss: 0.6233 - val_accuracy: 0.7927
Epoch 2/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0835 - accuracy: 0.9670 - val_loss: 0.8494 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 6s 30ms/step - loss: 0.0704 - accuracy: 0.9727 - val_loss: 0.7927 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 9s 43ms/step - loss: 0.0681 - accuracy: 0.9736 - val_loss: 1.0936 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0520 - accuracy: 0.9775 - val_loss: 1.2349 - val_accuracy: 0.7756


In [71]:
model_3.summary()

Model: "GRU_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_______________________________________________

In [72]:
# Evaluate model
model_3.evaluate(test_sentences, test_labels)

24/24 [==============================] - 0s 6ms/step - loss: 1.2349 - accuracy: 0.7756


[1.2349207401275635, 0.7755905389785767]

In [73]:
pred_probs = model_3.predict(test_sentences)
pred_probs[:10]

24/24 [==============================] - 1s 5ms/step


array([[3.7168316e-04],
       [6.9467187e-01],
       [9.9987763e-01],
       [4.9690984e-02],
       [1.6377565e-04],
       [9.9984676e-01],
       [8.7350720e-01],
       [9.9997342e-01],
       [9.9992543e-01],
       [9.8510522e-01]], dtype=float32)

In [74]:
preds_3  = tf.squeeze(tf.round(pred_probs))
preds_3[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [75]:
model_3_results = model_evaluation(test_labels, preds_3)
model_3_results

,0
accuracy score,0.775591
precision score,0.792079
recall score,0.689655
f1-score,0.737327


In [76]:
# Let's compare the model 3 results with baseline model
np.array(list(results.values)) < np.array(list(model_3_results.values))

array([[False],
       [False],
       [ True],
       [ True]])

## Model 4: Bidirectional RNN model

In [77]:
# Build a bidirectional RNN in TensorFlow
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x) # return_sequences=True required for stacking RNN layers
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="Bidirectional_Model")

In [78]:
model_4.summary()

Model: "Bidirectional_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378

In [79]:
# Compile model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit model
meta_4 = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(test_sentences, test_labels),
                              callbacks=[tensorboard_callback(dir_name="model_logs",
                                                                     experiment_name="model_4_bidirectional")])

Saving TensorBoard logfiles to: model_logs/model_4_bidirectional/31012023-102049
Epoch 1/5
215/215 [==============================] - 13s 42ms/step - loss: 0.1051 - accuracy: 0.9691 - val_loss: 1.0107 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 8s 38ms/step - loss: 0.0520 - accuracy: 0.9772 - val_loss: 1.0339 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 8s 38ms/step - loss: 0.0475 - accuracy: 0.9781 - val_loss: 1.2394 - val_accuracy: 0.7585
Epoch 4/5
215/215 [==============================] - 8s 37ms/step - loss: 0.0433 - accuracy: 0.9796 - val_loss: 1.2835 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 10s 49ms/step - loss: 0.0412 - accuracy: 0.9799 - val_loss: 1.4425 - val_accuracy: 0.7677


In [80]:
# Make predictions with our bidirectional model
model_4_pred_probs = model_4.predict(test_sentences)
model_4_pred_probs[:10], model_4_pred_probs.shape

24/24 [==============================] - 1s 9ms/step


(array([[1.4736375e-04],
        [6.3899648e-01],
        [9.9993587e-01],
        [1.4948522e-01],
        [1.6877755e-05],
        [9.9672180e-01],
        [9.3414444e-01],
        [9.9997044e-01],
        [9.9993926e-01],
        [9.9789953e-01]], dtype=float32), (762, 1))

In [81]:
# Convert pred probs to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [82]:
# Calculate model 4 results
model_4_results = model_evaluation(y_true=test_labels,
                                    y_pred=model_4_preds)
model_4_results

,0
accuracy score,0.767717
precision score,0.780328
recall score,0.683908
f1-score,0.728943


In [83]:
# Let's compare the model 3 results with baseline model
np.array(list(results.values)) < np.array(list(model_4_results.values))

array([[False],
       [False],
       [ True],
       [False]])

## Model 5: Conv1D

We've seen before how convolutional neural networks can be used for images but they can also be used for text.

Previously we've used the layer Conv2D (which is great for images with (height, width)).

But if we want to use convolutional layers for sequences (e.g. text) we need to use Conv1D: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D

For more of a deep dive into what goes on behind the scenes in a CNN for text (or sequences) see the paper: https://arxiv.org/abs/1809.08037

In [84]:
# Let's do a little test to see what things look like through the eyes of a Conv1D layer
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sentence into embedding
conv_1d_layer = layers.Conv1D(filters=32,
                              kernel_size=5, # setting this to 5 means it'll look at 5 words at a time, 3 would mean 3 words at a time
                              activation="relu")
conv_1d_output = conv_1d_layer(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
print(f"Embedding output shape: {embedding_test.shape}")
print(f"Conv1D output shape: {conv_1d_output.shape}")
print(f"Max pool output shape: {max_pool_output.shape}")

Embedding output shape: (1, 15, 128)
Conv1D output shape: (1, 11, 32)
Max pool output shape: (1, 32)


In [85]:
# Let's see the outputs of each layer
print(f"Embedding output: {embedding_test}")
print(f"Conv1D output: {conv_1d_output}")
print(f"Max pool output: {max_pool_output}")

Embedding output: [[[ 0.05838627  0.0103887  -0.0571257  ...  0.00102411 -0.01798349
    0.07335299]
  [-0.00630571  0.03083124 -0.04284771 ... -0.0058936   0.01802954
    0.01129492]
  [-0.04695208  0.00833625 -0.04635035 ...  0.00623493 -0.02667677
   -0.02199599]
  ...
  [ 0.0132613  -0.01198762 -0.03006701 ...  0.03974669 -0.04194493
    0.03520553]
  [ 0.0132613  -0.01198762 -0.03006701 ...  0.03974669 -0.04194493
    0.03520553]
  [ 0.0132613  -0.01198762 -0.03006701 ...  0.03974669 -0.04194493
    0.03520553]]]
Conv1D output: [[[0.06632335 0.         0.01679369 0.03121031 0.         0.04587021
   0.00286431 0.         0.07983069 0.0365255  0.         0.
   0.         0.0066168  0.02439709 0.08394636 0.01454431 0.
   0.         0.02708806 0.01999771 0.         0.0241394  0.
   0.         0.         0.02978971 0.01359272 0.         0.
   0.01378036 0.        ]
  [0.         0.         0.09101332 0.         0.         0.
   0.         0.         0.         0.         0.         0.


In [86]:
# Create 1-dimensional CNN to model sequences

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid", name="output_layer")(x)
model_5 = tf.keras.Model(inputs, outputs, name="conv1D_Model")

model_5.summary()

Model: "conv1D_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 output_layer (Dense)        (None, 1)                

In [87]:
BATCH_SIZE = 32
SAVE_DIR = "model_logs"

In [88]:
# Compile model
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
meta_5 = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              batch_size=BATCH_SIZE,
                              validation_data=(test_sentences, test_labels),
                              callbacks=[tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_5_conv1d")])

Saving TensorBoard logfiles to: model_logs/model_5_conv1d/31012023-102139
Epoch 1/5
215/215 [==============================] - 5s 19ms/step - loss: 0.1342 - accuracy: 0.9623 - val_loss: 0.8815 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 4s 21ms/step - loss: 0.0773 - accuracy: 0.9718 - val_loss: 0.9896 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 4s 21ms/step - loss: 0.0622 - accuracy: 0.9752 - val_loss: 1.1083 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0550 - accuracy: 0.9784 - val_loss: 1.1871 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0509 - accuracy: 0.9784 - val_loss: 1.1833 - val_accuracy: 0.7625


In [89]:
# Make predictions with our 1D CNN
model_5_pred_probs = model_5.predict(test_sentences)
model_5_pred_probs[:5], model_5_pred_probs.shape

24/24 [==============================] - 0s 4ms/step


(array([[2.0963979e-01],
        [4.3645221e-01],
        [9.9995774e-01],
        [4.9758747e-02],
        [1.6639711e-07]], dtype=float32), (762, 1))

In [90]:
# Convert model 5 pred probs to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [91]:
# Calculate results for model 5
model_5_results = model_evaluation(y_true=test_labels,
                                    y_pred=model_5_preds)
model_5_results

,0
accuracy score,0.762467
precision score,0.760125
recall score,0.701149
f1-score,0.729447


In [92]:
# Let's compare the model 5 results with baseline model
np.array(list(results.values)) < np.array(list(model_5_results.values))

array([[False],
       [False],
       [ True],
       [False]])

## Model 6: TensorFlow Hub Pretrained Sentence Encoder

Now we've built a few of our own models, let's try and use transfer learning for NLP, specifically using TensorFlow Hub's Universal Sentence Encoder: https://tfhub.dev/google/universal-sentence-encoder/4

See how the USE was created here: https://arxiv.org/abs/1803.11175

📖 **Resource:** TensorFlow Hub is a great resource for many pretrained models but HuggingFace is also another incredible resource for many pretrained NLP models (using HuggingFace model is beyond the scope of this course but it is definitely something you should be familiar with in the NLP space): https://huggingface.co/models

In [93]:
sample_sentence = "There's a flood in my street!"

In [94]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence,
                       "When you can the universal sentence encoder on a sentence, it turns it into numbers."])
print(embed_samples[0][:50])

tf.Tensor(
[-0.01157025  0.02485911  0.02878051 -0.012715    0.03971541  0.08827761
  0.02680988  0.05589838 -0.01068731 -0.00597293  0.00639321 -0.01819516
  0.00030816  0.09105889  0.05874645 -0.03180629  0.01512474 -0.05162925
  0.00991366 -0.06865345 -0.04209306  0.0267898   0.03011009  0.00321065
 -0.00337968 -0.04787356  0.0226672  -0.00985927 -0.04063615 -0.01292093
 -0.04666382  0.05630299 -0.03949255  0.00517682  0.02495827 -0.07014439
  0.0287151   0.0494768  -0.00633978 -0.08960193  0.02807119 -0.00808364
 -0.01360601  0.05998649 -0.10361788 -0.05195372  0.00232958 -0.02332531
 -0.03758106  0.03327729], shape=(50,), dtype=float32)


In [95]:
embed_samples[0].shape

TensorShape([512])

In [96]:
# Create a Keras Layer using the USE pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [97]:
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
  sentence_encoder_layer,
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid", name="output_layer")                               
], name="model_6_USE")

# Compile
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_1 (Dense)             (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [98]:
# Train a classifier on top of USE pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(test_sentences, test_labels),
                              callbacks=[tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentence_encoder")])

Saving TensorBoard logfiles to: model_logs/tf_hub_sentence_encoder/31012023-102231
Epoch 1/5
215/215 [==============================] - 6s 15ms/step - loss: 0.5083 - accuracy: 0.7765 - val_loss: 0.4489 - val_accuracy: 0.8018
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4149 - accuracy: 0.8161 - val_loss: 0.4355 - val_accuracy: 0.8110
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4001 - accuracy: 0.8244 - val_loss: 0.4312 - val_accuracy: 0.8110
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3910 - accuracy: 0.8300 - val_loss: 0.4265 - val_accuracy: 0.8150
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3845 - accuracy: 0.8301 - val_loss: 0.4262 - val_accuracy: 0.8123


In [99]:
# Make predictions with USE TF Hub Model
model_6_pred_probs = model_6.predict(test_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 10ms/step


array([[0.17049474],
       [0.81020975],
       [0.9935479 ],
       [0.21317676],
       [0.75689644],
       [0.7677769 ],
       [0.9832552 ],
       [0.9841975 ],
       [0.9484878 ],
       [0.08447162]], dtype=float32)

In [100]:
# Convert prediction probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [101]:
# Calculate model 6 performance metrics
model_6_results = model_evaluation(y_true=test_labels,
                                    y_pred=model_6_preds)
model_6_results

,0
accuracy score,0.812336
precision score,0.831715
recall score,0.738506
f1-score,0.782344


## Model 7: TF Hub Pretrained USE but with 10% of training data

Transfer learning really helps when you don't have a large dataset.

To see how our model performs on a smaller dataset, let's replicate `model_6` except we'll train it on 10% of the data.

In [102]:
# ## NOTE: Making data splits like below leads to data leakage (model_7 trained on 10% data, outperforms model_6 trained on 100% data)
# ## DO NOT MAKE DATA SPLITS WHICH LEAK DATA FROM VALIDATION/TEST SETS INTO TRAINING SET 

# # Create subsets of 10% of the training data
# train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)
# # train_10_percent.head(), len(train_10_percent)
# train_sentences_10_percent = train_10_percent["text"].to_list()
# train_labels_10_percent = train_10_percent["target"].to_list()
# len(train_sentences_10_percent), len(train_labels_10_percent)

> 🔑 **Note:** Be *very* careful when creating training/val/test splits that you don't leak data across the datasets, otherwise your model evaluation metrics will be wrong. If something looks too good to be true (a model trained on 10% of data outperforming the same model trained on 100% of data) trust your gut and go back through to find where the error may lie.

In [ ]:
# Making a better dataset split (no data leakage)
train_10_percent_split = int(0.1 * len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]

In [ ]:
# Check the number of each label in the updated training data subset
pd.Series(np.array(train_labels_10_percent)).value_counts()

0    406
1    279
dtype: int64

In [ ]:
# Check the number of targets in our subset of data
train_data_shuffled["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
# Let's build a model the same as model_6
# model_7 = tf.keras.models.clone_model(model_6)
model_7 = tf.keras.Sequential([
  sentence_encoder_layer,
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid", name="output_layer")                               
], name="model_7_USE")

# Compile model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary (will be same as model_6)
model_7.summary()

Model: "model_7_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_5 (Dense)              (None, 64)                32832     
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
# Fit the model to the 10% training data subsets
model_7_history = model_7.fit(train_sentences_10_percent,
                              train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentence_encoder_10_percent_correct_split")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder_10_percent_correct_split/20210414-060159
Epoch 1/5
22/22 [==============================] - 3s 101ms/step - loss: 0.6793 - accuracy: 0.6465 - val_loss: 0.6444 - val_accuracy: 0.7480
Epoch 2/5
22/22 [==============================] - 1s 29ms/step - loss: 0.6032 - accuracy: 0.8443 - val_loss: 0.5847 - val_accuracy: 0.7730
Epoch 3/5
22/22 [==============================] - 1s 29ms/step - loss: 0.5272 - accuracy: 0.8262 - val_loss: 0.5307 - val_accuracy: 0.7848
Epoch 4/5
22/22 [==============================] - 1s 28ms/step - loss: 0.4612 - accuracy: 0.8385 - val_loss: 0.5019 - val_accuracy: 0.7730
Epoch 5/5
22/22 [==============================] - 1s 29ms/step - loss: 0.4213 - accuracy: 0.8317 - val_loss: 0.4878 - val_accuracy: 0.7769


In [ ]:
# Make predictions with the model trained on 10% of the data
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

array([[0.2083207 ],
       [0.6036797 ],
       [0.92795014],
       [0.37632278],
       [0.5633504 ],
       [0.6833203 ],
       [0.8904111 ],
       [0.80405366],
       [0.85651606],
       [0.15267055]], dtype=float32)

In [ ]:
# Turn pred probs into labels
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
# Evalaute model 7 predictions
model_7_results = calculate_results(y_true=val_labels,
                                    y_pred=model_7_preds)
model_7_results

{'accuracy': 77.69028871391076,
 'f1': 0.7758759837011532,
 'precision': 0.7769803060533722,
 'recall': 0.7769028871391076}